In [37]:
import re
import db.un as db_un
from helpers.skus import str_to_dict, get_dosage

In [3]:
skus = db_un.get_en_skus()

In [19]:
dosages = dict() # {[id]: value} 

for un_id, data_str, row_num in skus:
    sku_data = str_to_dict(data_str)
    man_forms = sku_data['man_forms']
    for man_form in man_forms:
        dosage_form = man_form['dosage_form']
        name = dosage_form['name']
        dosage_id = dosage_form['id']
        if dosage_id not in dosages:
            dosages[dosage_id] = dict(name=name)
        if 'docs' not in dosages[dosage_id]:
            dosages[dosage_id]['docs'] = set()
        docs = dosages[dosage_id]['docs']
        docs.add(row_num)
#         dosages[dosage_id]
#         print(row_num, un_id, name)
#     print('\n\n')
#     dosage = get_dosage(sku_data)
#     dosages.add(dosage)
# result = ''
#   for item in sku_data['man_forms']:
#     result = result + item['dosage_form']['name']
#   return result


# m = 0
# m = max(m, len(man_forms))
# print(m)
# => 5


In [35]:
dosages_items = list(dosages.items())[:1000]
dosages_items[245]

('3de61dcf-2b73-44d2-a3ed-f0fa6a4f6936',
 {'name': 'порошок для приготовления …',
  'docs': {6614, 43900, 43907, 43914, 86892, 86908}})

In [76]:
# one_let = set()
# two_let = set()
three_let = set()
for row, value in three:
    print('{} `{}` [{}]'.format(row, value, dosages_items[row][1]['name']))
    three_let.add(value)
three_let

3 `для` [раствор для внутримышечного и подкожного введения]
6 `для` [капли назальные [для детей]]
8 `для` [гель для наружного применения]
9 `для` [раствор для инъекций и ингаляций]
12 `для` [порошок для приготовления раствора для внутримышечного введения]
12 `для` [порошок для приготовления раствора для внутримышечного введения]
14 `для` [эмульсия для наружного применения]
16 `рта` [таблетки, диспергируемые в полости рта]
19 `для` [раствор для внутривенного и подкожного введения]
20 `для` [раствор для внутримышечного введения]
22 `для` [порошок для приготовления суспензии для приёма внутрь]
22 `для` [порошок для приготовления суспензии для приёма внутрь]
23 `для` [концентрат для приготовления раствора для инфузий]
23 `для` [концентрат для приготовления раствора для инфузий]
24 `для` [раствор для внутривенного и внутримышечного введения]
25 `для` [раствор для инъекций]
26 `для` [порошок для приготовления раствора для внутривенного и внутримышечного введения]
26 `для` [порошок для пригот

{'без', 'газ', 'для', 'лак', 'рта', 'сок', 'чая', '№00'}

In [79]:
## 268 `el` [вспомогательное вещество-капсулы желатиновые твердые №0 el]
# 600 `е1` [вспомогательное вещество-капсулы целлюлозные №0 е1]
for row in dosages_items[529][1]['docs']:
    print(skus[row][0])

4e65dd8b-1237-4bbd-928d-0417727a3f2d
e0f4dbe9-f9c4-4999-a988-9634af2a5642


In [78]:
dosages_items[529]

('bc479032-5c43-4f32-a98f-e020fcb77f1d',
 {'name': 'вспомогательное вещество-капсулы целлюлозные №00',
  'docs': {46077, 47318}})

In [77]:
two_let

{'el', 'е1', 'на', 'со', '№0', '№1', '№2', '№3', '№4'}

In [60]:
one = list()
two = list()
three = list()
single_stop = {'в', 'и', 'с'}
dual_stop = {'на', 'со'}
for i, (form_id, form_dict) in enumerate(dosages_items):
    name = form_dict['name']
    docs = form_dict['docs']
    bag_of_words = re.split(
        r"\+|\|\*| |\n|\(|\)|\-|\[|\]|'|\"|\t|«|»|&|;|:|,",
        name
    )
    bow = list()
    for word in bag_of_words:
        if len(word) == 0: # ""
            continue
        # {'2', '4', 'в', 'и', 'с', '…'}
        # '4' 661 раствор для гемодиализа [калий 4 ммоль/л] ['раствор', 'для', 'гемодиализа', 'калий', '4', 'ммоль/л']
        # '2' 662 раствор для гемодиализа [калий 2 ммоль/л]
        # 245 `…` [порошок для приготовления …]
        if len(word) == 1:
            one.append((i, word))
            if word in single_stop:
                continue
        # {'el', 'е1', 'на', 'со', '№0', '№1', '№2', '№3', '№4'}
        # 268 `el` [вспомогательное вещество-капсулы желатиновые твердые №0 el]
        # 600 `е1` [вспомогательное вещество-капсулы целлюлозные №0 е1]
        if len(word) == 2:
            two.append((i, word))
            if word in dual_stop:
                continue
        # 529 `№00` [вспомогательное вещество-капсулы целлюлозные №00]
        if len(word) == 3:
            three.append((i, word))


        bow.append(word)
#     bow = [word for word in bag_of_words if len(word) > 0]
    print('[{}] [{}]'.format(i, len(docs)))
    print('{}\n{}\n{}\n\n'.format(name, bag_of_words, bow))

[0] [1145]
капли глазные
['капли', 'глазные']
['капли', 'глазные']


[1] [14]
таблетки ретард покрытые плёночной оболочкой
['таблетки', 'ретард', 'покрытые', 'плёночной', 'оболочкой']
['таблетки', 'ретард', 'покрытые', 'плёночной', 'оболочкой']


[2] [24608]
таблетки
['таблетки']
['таблетки']


[3] [109]
раствор для внутримышечного и подкожного введения
['раствор', 'для', 'внутримышечного', 'и', 'подкожного', 'введения']
['раствор', 'для', 'внутримышечного', 'и', 'подкожного', 'введения']


[4] [11359]
капсулы
['капсулы']
['капсулы']


[5] [30258]
таблетки, покрытые плёночной оболочкой
['таблетки', '', 'покрытые', 'плёночной', 'оболочкой']
['таблетки', 'покрытые', 'плёночной', 'оболочкой']


[6] [6]
капли назальные [для детей]
['капли', 'назальные', '', 'для', 'детей', '']
['капли', 'назальные', 'для', 'детей']


[7] [641]
таблетки пролонгированного действия, покрытые пленочной оболочкой
['таблетки', 'пролонгированного', 'действия', '', 'покрытые', 'пленочной', 'оболочкой']
['таблетки'

In [6]:
len(dosages)

776